In [32]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [33]:
import json
from pathlib import Path
import cv2

### images

In [34]:
path = Path('/home/zhangyu/mmdetection/data/compe/val2017')

In [35]:
id_tmp = 2019000001
images_list = []
for i in path.glob('*'):
    tmp_dict = {}
    img = cv2.imread(str(i))
    tmp_dict['file_name'] = i.stem
    tmp_dict['width'] = img.shape[1]
    tmp_dict['height'] = img.shape[0]
    tmp_dict['id'] = id_tmp
    id_tmp = id_tmp + 1
    images_list.append(tmp_dict)

### annotations

In [214]:
txt_path_pre = '/home/zhangyu/mmdetection/data/compe/source/val/labelTxt/labelTxt/'
## 加入标签
categories_tmp = {}
annotations = []
categories_tmp_index = 0
anno_dict_index = 0
for img_dict in images_list:
    txt_path = txt_path_pre + img_dict['file_name'].split('.')[0]+'.txt'
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        lines = list(set(lines))
        if '\n' in lines:
            lines.remove('\n')
        for line in lines:
            anno_dict = {}
            if 'gsd' in line or 'imagesource' in line:
                pass
            else:
                my_list = line.split(' ')
                if my_list[-2] not in categories_tmp:
                    categories_tmp[my_list[-2]] = categories_tmp_index
                    categories_tmp_index+=1
                
                segmentation = [float(i) for i in my_list[0:8]]
                x_min = np.array(segmentation[::2]).min()
                x_max = np.array(segmentation[::2]).max()
                y_min = np.array(segmentation[1::2]).min()
                y_max = np.array(segmentation[1::2]).max()
                anno_dict['segmentation'] = [segmentation]
                anno_dict['area'] = (x_max-x_min)*(y_max-y_min)
                anno_dict['iscrowd'] = 0
                anno_dict['image_id'] = img_dict['id']
                anno_dict['bbox'] = [x_min,y_min,x_max,y_max]
                anno_dict['category_id'] = categories_tmp[my_list[-2]]
                anno_dict['id'] = anno_dict_index+1
                anno_dict_index +=1
                annotations.append(anno_dict)

In [4]:
my_json = {}
my_json['images'] = images_list
my_json['type'] = 'instances'

with open('/home/zhangyu/mmdetection/data/compe/annotations/instances_val2017.json','w') as f:
    load_dict = json.dump(my_dict,f)

In [110]:
with open('/home/zhangyu/mmdetection/data/coco/annotations/instances_val2017.json','r') as f:
    coco_dict = json.load(f)

In [8]:
coco_dict.keys()

dict_keys(['images', 'type', 'annotations', 'categories'])

In [126]:
txt_path = '/home/zhangyu/mmdetection/data/compe/source/val/labelTxt/labelTxt/P2129.txt'
with open(txt_path, 'r') as f:
    test = f.readlines()